In [ ]:
import keras

# エラー
# keras_model = keras.models.load_model('./SRCNN_no-tf_2.h5')

In [ ]:
import numpy as np
from keras import backend as K

def psnr(y_true, y_pred):
    return -10 * K.log(
        K.mean(K.flatten((y_true - y_pred))**2)
    ) / np.log(10)

In [ ]:
keras_model = keras.models.load_model('./SRCNN_no-tf_2.h5', 
                                      custom_objects={'psnr':psnr})

In [ ]:
print(keras_model.input_shape)

In [ ]:
keras_model.summary()

In [ ]:
# 変換しようとしてみる
from coremltools.converters import keras as converter

# mlmodel = converter.convert(keras_model) # エラー

In [ ]:
# 入力シェイプを指定して変換
mlmodel = converter.convert(keras_model,
                            input_name_shape_dict = {'input1':[None,200,200,3]},
                            image_input_names = ['input1'])

In [ ]:
spec = mlmodel.get_spec()
print(spec.description)

In [ ]:
# 出力の型を変更する
import coremltools
from coremltools.proto import FeatureTypes_pb2 as ft

image_type = spec.description.output[0].type.imageType
image_type.colorSpace = ft.ImageFeatureType.ColorSpace.Value('RGB')
image_type.width = 200
image_type.height = 200

In [ ]:
print(spec.description)

In [ ]:
# .mlmodelファイルとして保存
coremltools.models.utils.save_spec(spec, './srcnn_fixed_shapes.mlmodel')

In [ ]:
from coremltools.models.neural_network import flexible_shape_utils

flexible_shape_utils.can_allow_multiple_input_shapes(spec)

In [ ]:
flexible_shape_utils.get_allowed_shape_ranges(spec)

In [ ]:
size_range = flexible_shape_utils.NeuralNetworkImageSizeRange()
size_range.add_width_range((100, 400))
size_range.add_height_range((100, 400))
flexible_shape_utils.update_image_size_range(spec,                                            
                                             feature_name='input1',
                                             size_range=size_range)

In [ ]:
print(spec.description)

In [ ]:
flexible_shape_utils.update_image_size_range(spec,
                                             feature_name='output1',                                            
                                             size_range=size_range)

In [ ]:
print(spec.description)

In [ ]:
print(spec.specificationVersion)

In [ ]:
coremltools.utils.save_spec(spec, "./srcnn_flexible_shapes.mlmodel")